In [1]:
#!/usr/bin/env python
import pandas as pd
import torch
import tensorqtl
from tensorqtl import pgen, cis
import time
import os
import os.path as op
import sys
import glob
import re
print(f'PyTorch {torch.__version__}')
#print(torch.__version__)
print('CUDA available: {} ({})'.format(torch.cuda.is_available(), torch.cuda.get_device_name(torch.cuda.current_device())))
print(f'Pandas {pd.__version__}')
print(f'tensorqtl {tensorqtl.__version__}')

outdir='eqtl_out'
if not os.path.exists(outdir):
    os.makedirs(outdir)

PyTorch 2.5.1+cu124
CUDA available: True (NVIDIA GeForce RTX 4070 Ti SUPER)
Pandas 2.2.3
tensorqtl 1.0.10


In [2]:
## ---- CHANGE HERE --------------
in_dir='eqtl_inputs' ## directory path where the tensorqtl input files can be found:
                     ## dsname.gene.expr.bed.gz and dsname.covars.txt
dsname='claustrum' ## dataset name, this is the prefix for input and output file names
plink='genotypes/claustrum_maf05' ## plink prefix for genotype data
#plink='genotypes/mdd_maf01'
mapping_file=None ## comment the line below if genotype IDs are the same with RNAseq sample IDs.
#mapping_file='genoID2rnaID.tab' # genotype IDs will be changed to their RNAseq mappings in the 2nd column
#mapping_file='genotypes/mdd_geno2rna.tab'
## if not None, mapping_file must be the same with the one used for 02_prep_tensorQTL_RSE_input.R
col_map = None
if mapping_file is not None and op.exists(mapping_file):
    print(f"mapping genotype IDs to RNAseq sample IDs based on mapping file: {mapping_file}")
    mapping_df = pd.read_csv(mapping_file, sep="\t", header=None, names=["gt_id", "r_id"])
    col_map = dict(zip(mapping_df['gt_id'], mapping_df['r_id']))
if op.exists(plink+'.pgen'): # checks for plink2 pgen file for genotype data
    print("Plink2 pgen file found.")
else:
    raise Exception("Plink2 "+plink+".pgen not found!")

exprfiles=glob.glob(in_dir+'/'+dsname+'.*.expr.bed.gz')
features = [os.path.basename(file).split('.')[1] for file in exprfiles]
exprdict = {os.path.basename(file).split('.')[1]: file for file in exprfiles}
for feat in features:
    expres = exprdict[feat]
    covar=expres.replace("expr.bed.gz", "covars.txt")
    if not op.exists(covar):
       raise FileNotFoundError(error_message)("Covars file "+covar+" not found!")
print("Features found: ",features)
print(" Change the `features` array below to set the features to be processed:")
## CHANGE here and uncomment, if needed, in order to select the features to process:
features = ['gene']
print("Features to process: ", features)

Plink2 pgen file found.
Features found:  ['gene']
 Change the `features` array below to set the features to be processed:
Features to process:  ['gene']


In [3]:
#pr = genotypeio.PlinkReader(plink)
## use plink2 pgen format
pgr = pgen.PgenReader(plink)
genotype_df = pgr.load_genotypes()
#variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]
variant_df = pgr.variant_df
print("Genotype dimensions: ", end='')
print(genotype_df.shape)
# if a mapping file is given
# Check if all genotype_df column names are in the mapping file
#all_columns_found = genotype_df.columns.isin(mapping_df["current_column"]).all()

if col_map is not None:
  # Rename columns using the dictionary
  genotype_df.rename(columns=col_map, inplace=True)
## Now genotype_df contains the updated column names
#print(genotype_df.iloc[:5, :7])  # Display the first few rows



Genotype dimensions: (6026215, 60)


In [4]:
def fixBrNums(col):
    return re.sub(r'^Br(\d\d\d)$', r'Br0\1', col)

## fix Brnums with 3 digits
genotype_df.columns = [fixBrNums(col) for col in genotype_df.columns]


##  etc. Fix chromosomes (add the "chr" prefix) if needed:
if not variant_df.chrom.iloc[0].startswith('chr'):
   variant_df.chrom = [ 'chr' + chrom for chrom in variant_df.chrom]
## select chromosomes - to make sure we have the same chromosomes in our data for each expression dataset
variant_chrom = set(variant_df.chrom)

for feat in features:
    print(f" Processing feature: {feat}")
    tag = dsname+'.'+feat
    expres = exprdict[feat]
    covar=expres.replace("expr.bed.gz", "covars.txt")
    covariates_df = pd.read_csv(covar, sep='\t', index_col=0).T
    ## fix Brnums with 3 digits
    covariates_df.index = [fixBrNums(idx) for idx in covariates_df.index]

    print("Covariates dim:", end='')
    print(covariates_df.shape)
    #print(covariates_df.iloc[:5, :5])
    phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expres)
    print("Phenotype dimensions:", end='')
    print(phenotype_df.shape)
    ## fix Brnums with 3 digits
    phenotype_df.columns = [fixBrNums(col) for col in phenotype_df.columns]


    ## use the same chromosome set
    express_chrom = set(phenotype_pos_df.chr)
    assert len(variant_chrom.intersection(express_chrom))>0
    if express_chrom - variant_chrom:
        chrom_filter = phenotype_pos_df.chr.isin(variant_chrom)
        if (len(chrom_filter)<phenotype_df.shape[0]):
          phenotype_df = phenotype_df[chrom_filter]
          phenotype_pos_df = phenotype_pos_df[chrom_filter]
    ## make sure we keep only the genotypes for the same expression samples
    cols=phenotype_df.columns.tolist()
    gcols=genotype_df.columns.tolist()
    ## which genotypes are in the expression data? print them here if any are missing and stop the process
    missing_geno = set(cols) - set(gcols)
    if missing_geno:
        print(f"Genotypes missing for {len(missing_geno)} samples: {missing_geno}")
        raise RuntimeError("Missing genotypes detected. Aborting...")
    ## show if any genotypes are not in the expression data:
    missing_expr = set(gcols) - set(cols)
    if missing_expr:
        print(f"These genotypes have no expression data given: {missing_expr}")
    geno_df=genotype_df.loc[:, cols]
    ## run tensorQTL:
    cis.map_nominal(geno_df, variant_df, phenotype_df, phenotype_pos_df, prefix = tag, covariates_df= covariates_df,
                maf_threshold=0.05, window=500000, output_dir= outdir, verbose=False)
print("All done.")

 Processing feature: gene
Covariates dim:(60, 8)
Phenotype dimensions:(25143, 60)
cis-QTL mapping: nominal associations for all variant-phenotype pairs
  * 60 samples
  * 25143 phenotypes
  * 8 covariates
  * 6026215 variants
  * applying in-sample 0.05 MAF filter
  * cis-window: ±500,000
  * checking phenotypes: 25143/25143
    ** dropping 144 phenotypes without variants in cis-window
  * Computing associations
    Mapping chromosome chr1
    time elapsed: 0.05 min
    * writing output
    Mapping chromosome chr2
    time elapsed: 0.12 min
    * writing output
    Mapping chromosome chr3
    time elapsed: 0.18 min
    * writing output
    Mapping chromosome chr4
    time elapsed: 0.22 min
    * writing output
    Mapping chromosome chr5
    time elapsed: 0.26 min
    * writing output
    Mapping chromosome chr6
    time elapsed: 0.31 min
    * writing output
    Mapping chromosome chr7
    time elapsed: 0.36 min
    * writing output
    Mapping chromosome chr8
    time elapsed: 0.40 m

In [5]:
## run permutation-based mapping with map_cis (takes longer)
## cis.map_cis is LD aware and it only outputs the most significant eQTL per gene
cis_df = cis.map_cis(geno_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df)
tensorqtl.calculate_qvalues(cis_df, qvalue_lambda=0.85, fdr=0.05)
outfile = os.path.join(outdir, f"{tag}.map_cis.tab.gz")
cis_df.to_csv(outfile, sep='\t', compression='gzip', index=True)

cis-QTL mapping: empirical p-values for phenotypes
  * 60 samples
  * 25143 phenotypes
  * 8 covariates
  * 6026215 variants
  * cis-window: ±1,000,000
  * checking phenotypes: 25143/25143
    ** dropping 93 phenotypes without variants in cis-window
  * computing permutations
    processing phenotype 25050/25050
  Time elapsed: 8.81 min
done.


In [24]:
print(type(cis_df))

<class 'pandas.core.frame.DataFrame'>
